<a href="https://colab.research.google.com/github/pranjal272/HDSC22/blob/main/StageB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import pandas as pd
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

In [122]:
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [123]:
#rename columns
column_names={'T1': 'Temperature in kitchen area','RH_1': 'Humidity in kitchen area', 'T2':'Temperature in living room area', 'RH_2': 'Humidity in living room area', 'T3': 'Temperature in laundry room area','RH_3':'Humidity in laundry room area', 'T4': 'Temperature in office room', 
'RH_4': 'Humidity in office room',
'T5':'Temperature in bathroom',
'RH_5': 'Humidity in bathroom',
'T6': 'Temperature outside the building (north side)',
'RH_6': 'Humidity outside the building (north side)',
'T7': 'Temperature in ironing room' ,
'RH_7': 'Humidity in ironing room', 
'T8': 'Temperature in teenager room 2', 
'RH_8':'Humidity in teenager room 2', 
'T9':'Temperature in parents room', 
'RH_9':'Humidity in parents room', 
'T_out':'Temperature outside (from Chievres weather station)',
'Press_mm_hg':'Pressure (from Chievres weather station)',
'RH_out': 'Humidity outside (from Chievres weather station)',
'rv1': 'Random variable 1',
'rv2': 'Random variable 2'}
df = df.rename(columns=column_names)
df.head()

,date,Appliances,lights,Temperature in kitchen area,Humidity in kitchen area,Temperature in living room area,Humidity in living room area,Temperature in laundry room area,Humidity in laundry room area,Temperature in office room,Humidity in office room,Temperature in bathroom,Humidity in bathroom,Temperature outside the building (north side),Humidity outside the building (north side),Temperature in ironing room,Humidity in ironing room,Temperature in teenager room 2,Humidity in teenager room 2,Temperature in parents room,Humidity in parents room,Temperature outside (from Chievres weather station),Pressure (from Chievres weather station),Humidity outside (from Chievres weather station),Windspeed,Visibility,Tdewpoint,Random variable 1,Random variable 2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


dropping date and lights columns:

In [124]:
df = df.drop(columns = ['date', 'lights'])

In [126]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
#target: Appliances variable
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
features_df = normalised_df.drop(columns = ['Appliances'])
target_df = normalised_df['Appliances']


A linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6).

In [127]:
#defining new predictor and response variable
x = normalised_df[['Temperature in living room area']]
y = normalised_df['Temperature outside the building (north side)']

#split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

#model definition and fitting
simple_linear_reg_model = LinearRegression()
simple_linear_reg_model.fit(x_train, y_train)

#predictions
predicted_values = simple_linear_reg_model.predict(x_test)

In [128]:
#R^2 value in two d.p.
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.64

In [129]:
# linear model with all predictors
X_train, X_test, Y_train, Y_test = train_test_split(features_df, target_df, test_size = 0.3, random_state = 42)

#model definition and fitting
linear_model = LinearRegression()
linear_model.fit(X_train, Y_train)

#obtain predictions
yhat = linear_model.predict(X_test)

In [130]:
#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(Y_test, yhat)
round(mae, 2)

0.05

In [132]:
#Residual Sum of Squares (RSS)
import numpy as np
rss = np.sum(np.square(Y_test - yhat))
round(rss, 2) 

45.35

In [133]:
#Root Mean Square Error (RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(Y_test, yhat))
round(rmse, 3)

0.088

In [134]:
# coefficient of determination
from sklearn.metrics import r2_score
r2_score = r2_score(Y_test, yhat)
round(r2_score, 2)

0.15

Feature weights:

In [135]:
def get_weights_df(model, feat, col_name):
    
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

linear_model_weights = get_weights_df(linear_model, X_train, 'Linear_Model_Weight')
linear_model_weights = linear_model_weights.sort_values(by = 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,Humidity in living room area,-0.456698
1,Temperature outside (from Chievres weather sta...,-0.321860
2,Temperature in living room area,-0.236178
3,Temperature in parents room,-0.189941
4,Humidity in teenager room 2,-0.157595
5,Humidity outside (from Chievres weather station),-0.077671
6,Humidity in ironing room,-0.044614
7,Humidity in parents room,-0.039800
8,Temperature in bathroom,-0.015657
9,Temperature in kitchen area,-0.003281


Lowest weight for:Humidity in living room area (RH_2), Highest weight for: Humidity in kitchen area	(RH_1)

In [136]:
#ridge regression with alpha = 0.4

ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(X_train, Y_train)

#predictions
ridge_yhat = ridge_reg.predict(X_test)

In [137]:
#rmse on test set upto 3 decimal places
round(np.sqrt(mean_squared_error(Y_test, ridge_yhat)), 3)

0.088

In [138]:
#lasso regression with alpha = 0.001 and its feature weights
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(X_train, Y_train)

#predictions
lasso_yhat = lasso_reg.predict(X_test)
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

number of features having non-zero weights

In [139]:
len(lasso_weights_df[lasso_weights_df['Lasso_weight'] != 0])

4

New RMSE with the lasso regression:

In [140]:
#rmse on test set upto 3 decimal places
round(np.sqrt(mean_squared_error(Y_test, lasso_yhat)), 3)

0.094